<center><h1>Train Data Predictions</h1></center>

This is the final stage on this Titanic Competition Analysis. In this stage we will measure three predictive models and we are going to compare their average rate. The models to compare are:
    1. Logistic Regression by Scikit Learn
    2. Decision Tree Clasiffier by Scikit Learn
    3. Human Predictive Analysis based on Data Analysis
For the firstly two points on our list we are going to use train_master_table beacuse we don't need any previous probability to send for this machine learning models. But in case of Human Prediction we need to use our train_prob_table with all probabilities according with our hypothesis.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib

%matplotlib inline

In [2]:
df_train = pd.read_csv('train_master_table.csv')
df_train.drop(['Unnamed: 0'], axis = 'columns',inplace = True)
print(df_train.shape)

(712, 7)


In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,Embarked,FamMem
0,1,0,3,0,1,0,1
1,2,1,1,1,1,1,1
2,3,1,3,1,1,0,0
3,4,1,1,1,1,0,1
4,5,0,3,0,1,0,0


<b>Logistic Regression / Decision Tree</b>

In [4]:
surv = df_train['Survived']
params = df_train[['Pclass', 'Sex', 'Age', 'Embarked', 'FamMem']]

x_train, x_test, y_train, y_test = train_test_split(params, surv, test_size = 0.15, random_state=1)

clf = [(LogisticRegression(), 'Logistic Regression'),
       (DecisionTreeClassifier(), 'Decision Tree')]

for model , name in clf:
    # Train model
    model.fit(x_train, y_train)
    
    # Measure model
    surv_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, surv_pred)
    print(f'{name} Accuracy: {round(accuracy, 4)}')

Logistic Regression Accuracy: 0.7383
Decision Tree Accuracy: 0.7664


<b>Human Predictive Analysis</b>

In [5]:
original_df = pd.read_csv('train.csv')
original_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
prob_df = pd.read_csv('train_prob_table.csv')
prob_df.rename(columns={'Unnamed: 0':'Tag'} ,inplace = True)
prob_df.head()

,Tag,0 Stage,1 Stage,2 Stage,3 Stage
0,Men Rate,0.205298,NaN,NaN,NaN
1,Women Rate,0.752896,NaN,NaN,NaN
2,1st Class Men,NaN,0.396040,NaN,NaN
3,2nd Class Men,NaN,0.151515,NaN,NaN
4,3rd Class Men,NaN,0.150198,NaN,NaN


In [7]:
# Classification of passengers according to existing data
def find_sex(row, i):
    if row['Sex'][i] == 'male':
        return 'male'
    else:
        return 'female'
    
def find_class(row, i):
    if row['Pclass'][i] == 1:
        return 1
    elif row['Pclass'][i] == 2:
        return 2
    else:
        return 3
    
def find_age(row, i):
    if row['Age'][i] <= 15:
        return 'Child'
    elif (row['Age'][i]>15) & (row['Age'][i]<= 60):
        return 'Adult'
    else:
        return 'Elder'
    
def find_famMem(row, i):
    if row['SibSp'][i] != 0 or row['Parch'][i] != 0:
        return 'FamMem'
    else:
        return 'Alone'

In [8]:
# Create a list which contains all features for each passenger
def pass_list(row, stage, i):
    pass_list = []
    if stage == 0:
        pass_list.append(find_sex(row, i))
    elif stage == 1:
        pass_list.append(find_sex(row, i))
        pass_list.append(find_class(row, i))
    elif stage == 2:
        pass_list.append(find_sex(row, i))
        pass_list.append(find_class(row, i))        
        pass_list.append(find_age(row, i))
    elif stage == 3: 
        pass_list.append(find_sex(row, i))
        pass_list.append(find_class(row, i))        
        pass_list.append(find_age(row, i))
        pass_list.append(find_famMem(row, i))
    
    return pass_list

In [9]:
def identify_passenger_prob(pass_clasf, stage):
    if stage == 0:
        if pass_clasf[0] == 'male':
            return prob_df[prob_df['Tag']=='Men Rate']['0 Stage'][0]
        elif pass_clasf[0] == 'female':
            return prob_df[prob_df['Tag']=='Women Rate']['0 Stage'][1]
        
    if stage == 1:
        if (pass_clasf[0] == 'male') & (pass_clasf[1] == 1):
            return prob_df[prob_df['Tag']=='1st Class Men']['1 Stage'][2]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 2):
            return prob_df[prob_df['Tag']=='2nd Class Men']['1 Stage'][3]     
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 3):
            return prob_df[prob_df['Tag']=='3rd Class Men']['1 Stage'][4]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 1):
            return prob_df[prob_df['Tag']=='1st Class Women']['1 Stage'][5]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 2):
            return prob_df[prob_df['Tag']=='2nd Class Women']['1 Stage'][6]     
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 3):
            return prob_df[prob_df['Tag']=='3rd Class Women']['1 Stage'][7] 
        
    if stage == 2:
        if (pass_clasf[0] == 'male') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Child'):
            return prob_df[prob_df['Tag']=='1st Class Men Children']['2 Stage'][8]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Adult'):
            return prob_df[prob_df['Tag']=='1st Class Men Adults']['2 Stage'][9]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Elder'):
            return prob_df[prob_df['Tag']=='1st Class Men Elders']['2 Stage'][10]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Child'):
            return prob_df[prob_df['Tag']=='2nd Class Men Children']['2 Stage'][11]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Adult'):
            return prob_df[prob_df['Tag']=='2nd Class Men Adults']['2 Stage'][12]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Elder'):
            return prob_df[prob_df['Tag']=='2nd Class Men Elders']['2 Stage'][13]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Child'):
            return prob_df[prob_df['Tag']=='3rd Class Men Children']['2 Stage'][14]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Adult'):
            return prob_df[prob_df['Tag']=='3rd Class Men Adults']['2 Stage'][15]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Elder'):
            return prob_df[prob_df['Tag']=='3rd Class Men Elders']['2 Stage'][16]
        
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Child'):
            return prob_df[prob_df['Tag']=='1st Class Women Children']['2 Stage'][17]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Adult'):
            return prob_df[prob_df['Tag']=='1st Class Women Adults']['2 Stage'][18]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Elder'):
            return prob_df[prob_df['Tag']=='1st Class Women Elders']['2 Stage'][19]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Child'):
            return prob_df[prob_df['Tag']=='2nd Class Women Children']['2 Stage'][20]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Adult'):
            return prob_df[prob_df['Tag']=='2nd Class Women Adults']['2 Stage'][21]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Elder'):
            return prob_df[prob_df['Tag']=='2nd Class Women Elders']['2 Stage'][22]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Child'):
            return prob_df[prob_df['Tag']=='3rd Class Women Children']['2 Stage'][23]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Adult'):
            return prob_df[prob_df['Tag']=='3rd Class Women Adults']['2 Stage'][24]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Elder'):
            return prob_df[prob_df['Tag']=='3rd Class Women Elders']['2 Stage'][25]
    
    if stage == 3:
        # First class men
        if (pass_clasf[0] == 'male') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='1st Class Men Children Alone']['3 Stage'][26]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='1st Class Men Children FamMem']['3 Stage'][27]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='1st Class Men Adults Alone']['3 Stage'][28]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='1st Class Men Adults FamMem']['3 Stage'][29]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='1st Class Men Elders Alone']['3 Stage'][30]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='1st Class Men Elders FamMem']['3 Stage'][31]  
        
        # Second class men
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='2nd Class Men Children Alone']['3 Stage'][32]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='2nd Class Men Children FamMem']['3 Stage'][33]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='2nd Class Men Adults Alone']['3 Stage'][34]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='2nd Class Men Adults FamMem']['3 Stage'][35]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='2nd Class Men Elders Alone']['3 Stage'][36]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='2nd Class Men Elders FamMem']['3 Stage'][37]    
        
        # Third class men
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='3rd Class Men Children Alone']['3 Stage'][38]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='3rd Class Men Children FamMem']['3 Stage'][39]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='3rd Class Men Adults Alone']['3 Stage'][40]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='3rd Class Men Adults FamMem']['3 Stage'][41]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='3rd Class Men Elders Alone']['3 Stage'][42]
        elif (pass_clasf[0] == 'male') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='3rd Class Men Elders FamMem']['3 Stage'][43]  
        
        # First class women
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='1st Class Women Children Alone']['3 Stage'][44]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='1st Class Women Children FamMem']['3 Stage'][45]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='1st Class Women Adults Alone']['3 Stage'][46]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='1st Class Women Adults FamMem']['3 Stage'][47]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='1st Class Women Elders Alone']['3 Stage'][48]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 1) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='1st Class Women Elders FamMem']['3 Stage'][49]  
        
        # Second class women
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='2nd Class Women Children Alone']['3 Stage'][50]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='2nd Class Women Children FamMem']['3 Stage'][51]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='2nd Class Women Adults Alone']['3 Stage'][52]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='2nd Class Women Adults FamMem']['3 Stage'][53]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='2nd Class Women Elders Alone']['3 Stage'][54]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 2) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='2nd Class Women Elders FamMem']['3 Stage'][55]    
        
        # Third class women
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='3rd Class Women Children Alone']['3 Stage'][56]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Child') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='3rd Class Women Children FamMem']['3 Stage'][57]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='3rd Class Women Adults Alone']['3 Stage'][58]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Adult') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='3rd Class Women Adults FamMem']['3 Stage'][59]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'Alone'):
            return prob_df[prob_df['Tag']=='3rd Class Women Elders Alone']['3 Stage'][60]
        elif (pass_clasf[0] == 'female') & (pass_clasf[1] == 3) & (pass_clasf[2] == 'Elder') & (pass_clasf[3] == 'FamMem'):
            return prob_df[prob_df['Tag']=='3rd Class Women Elders FamMem']['3 Stage'][61]  

In [10]:
first_list = list(range(0, len(original_df)))
global_prob_list = []
for i in first_list:
    j = i+1  
    if original_df[i:j]['Sex'].isna()[i] == False:
        if original_df[i:j]['Pclass'].isna()[i] == False:
            if original_df[i:j]['Age'].isna()[i] == False:
                if (original_df[i:j]['SibSp'].isna()[i] == False) or (original_df[i:j]['Parch'].isna()[i] == False):
                    stage = 3
                    pass_clasf = pass_list(original_df[i:j], stage, i)
                    prob_surv = identify_passenger_prob(pass_clasf, stage)
                    global_prob_list.append(prob_surv)
                else:
                    stage = 2
                    pass_clasf = pass_list(original_df[i:j], stage, i)
                    prob_surv = identify_passenger_prob(pass_clasf, stage)
                    global_prob_list.append(prob_surv)                   
            else:
                stage = 1
                pass_clasf = pass_list(original_df[i:j], stage, i)
                prob_surv = identify_passenger_prob(pass_clasf, stage)
                global_prob_list.append(prob_surv)
        else:
            stage = 0
            pass_clasf = pass_list(original_df[i:j], stage, i)
            prob_surv = identify_passenger_prob(pass_clasf, stage)
            global_prob_list.append(prob_surv)
    else:
        print('ERROR!')

In [11]:
len(global_prob_list)

891

In [12]:
original_df['SurvProb'] = global_prob_list
original_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SurvProb
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.076923
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.979592
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.500000
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.979592
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.142857


In [38]:
def surv_or_died(x):
    if x >= 0.5:
        return 1
    if x < 0.5:
        return 0

original_df['SurvPred'] = original_df['SurvProb'].apply(lambda x : surv_or_died(x))
original_df[0:10]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SurvProb,SurvPred
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.076923,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.979592,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.500000,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.979592,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.142857,0
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,0.150198,0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,0.422222,0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,0.333333,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,0.358974,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,1.000000,1


In [39]:
correct_predic = list(np.where(original_df['Survived'] == original_df['SurvPred'])[0])
average_hda = len(correct_predic)/len(global_prob_list)
print(f'Human Prediction Accuracy: {round(average_hda, 4)}')

Human Prediction Accuracy: 0.8047


<h1><center>Conclusion</center></h1>

On these three notebooks we used graphs and probability computing to get a predictive model that could decide if a Titanic passenger survived or died based on our given dataset. Our hypothesis were analazed and modified with the aim to optimize the data prediction model. Finally, we probed three predictive models, Logistic Regression, Decision Tree and Human Data Prediction based on Data Analysis. The results were the following:
    1. Logistic Regression Accuracy: 0.7383
    2. Decision Tree Accuracy: 0.7664
    3. Human Prediction Accuracy: 0.8047
This a great score for humans :) and personally I am happy for this average, but it's worth remember that firstly two models in this list were calculated in less than 1.5 seconds, and for a human like me, this Prediction Model was made in about three days. So I think that this is a partial victory for humans (only considerating Logistic Regression and Decision Tree Models).